<a href="https://colab.research.google.com/github/DhruvBhatia0/Neuralink_Compression/blob/main/Test_audio_encoder.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
! pip install transformers

In [ ]:
import torch
from transformers import Wav2Vec2ForCTC, Wav2Vec2Processor
import soundfile as sf
import librosa
from google.colab import files
import io
import os

In [ ]:
model = Wav2Vec2ForCTC.from_pretrained("facebook/wav2vec2-base-960h")
processor = Wav2Vec2Processor.from_pretrained("facebook/wav2vec2-base-960h")


Some weights of the model checkpoint at facebook/wav2vec2-base-960h were not used when initializing Wav2Vec2ForCTC: ['wav2vec2.encoder.pos_conv_embed.conv.weight_g', 'wav2vec2.encoder.pos_conv_embed.conv.weight_v']
- This IS expected if you are initializing Wav2Vec2ForCTC from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing Wav2Vec2ForCTC from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of Wav2Vec2ForCTC were not initialized from the model checkpoint at facebook/wav2vec2-base-960h and are newly initialized: ['wav2vec2.encoder.pos_conv_embed.conv.parametrizations.weight.original0', 'wav2vec2.encoder.pos_conv_embed.conv.parametrizations.weight.original1', 'wav2vec2.masked_spec_embed']
You sho

In [ ]:
def process_audio(input_file, output_file):
    # Load the audio file
    audio_input, sample_rate = librosa.load(input_file, sr=16000)

    # Prepare the input
    input_values = processor(audio_input, sampling_rate=16000, return_tensors="pt").input_values

    # Encode and get logits
    with torch.no_grad():
        logits = model(input_values).logits

    # Print shape and sample of encoded values (logits)
    print("Encoded shape:", logits.shape)
    print("Encoded values (first few):", logits[0, :5, :5])

    # Get the predicted token ids
    predicted_ids = torch.argmax(logits, dim=-1)

    # Decode the token ids to text
    transcription = processor.batch_decode(predicted_ids)
    print("Transcription:", transcription[0])

    # Save the processed audio (in this case, it's just resampled to 16kHz)
    sf.write(output_file, audio_input, 16000)

In [ ]:
! pwd

/content


In [18]:
# Upload file
print("Please upload a WAV file:")
uploaded = files.upload()

# Get the filename of the uploaded file
input_filename = list(uploaded.keys())[0]

# Get the full path of the input file
input_path = os.path.abspath(input_filename)
print(f"Input file path: {input_path}")

# Process the uploaded file
output_filename = "processed_" + input_filename
output_path = os.path.abspath(output_filename)

process_audio(input_path, output_path)

print(f"Processing complete. The processed file is stored at: {output_path}")

Please upload a WAV file:


Saving Miley Cyrus Wrecking Ball.wav to Miley Cyrus Wrecking Ball.wav
Input file path: /content/Miley Cyrus Wrecking Ball.wav
Encoded shape: torch.Size([1, 284, 32])
Encoded values (first few): tensor([[  6.1521, -20.4519, -20.3337, -20.5293,  -0.3651],
        [  6.7082, -20.0554, -19.9180, -20.1087,  -0.1517],
        [  8.3720, -21.5332, -21.3336, -21.5324,   0.6390],
        [  7.2269, -21.4588, -21.2483, -21.4517,   0.0273],
        [  6.2981, -22.6422, -22.4655, -22.7251,   1.0749]])
Transcription: I CAME IN LAT GO E
Processing complete. The processed file is stored at: /content/processed_Miley Cyrus Wrecking Ball.wav
